**I. Perform any necessary preprocessing on the text, including converting to lower case, removing stop words, numbers or non-alphabetic characters, lemmatization.**

In [157]:
f = open('Alice.txt','r')
raw_text = f.read().replace('\n', '')
raw_text[:20]

'CHAPTER I.Down the R'

In [4]:
import re
import numpy as np
text = str(raw_text).lower() #to lower-case
text = re.sub("\d+", "", str(text)) #remove numbers
regex = re.compile('[^a-zA-Z]') #remove everything except letters
text = regex.sub(' ', text)
text

'chapter i down the rabbit holealice was beginning to get very tired of sitting by her sister on thebank  and of having nothing to do  once or twice she had peeped intothe book her sister was reading  but it had no pictures orconversations in it   and what is the use of a book   thought alice without pictures or conversations  so she was considering in her own mind  as well as she could  for thehot day made her feel very sleepy and stupid   whether the pleasure ofmaking a daisy chain would be worth the trouble of getting up andpicking the daisies  when suddenly a white rabbit with pink eyes ranclose by her there was nothing so  very  remarkable in that  nor did alice think itso  very  much out of the way to hear the rabbit say to itself   ohdear  oh dear  i shall be late    when she thought it over afterwards it occurred to her that she ought to have wondered at this  but at thetime it all seemed quite natural   but when the rabbit actually  took awatch out of its waistcoat pocket   an

In [15]:
import nltk
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [12]:
from nltk.tokenize import TreebankWordTokenizer, WhitespaceTokenizer
from nltk.corpus import stopwords

stop_words = stopwords.words("english")
tokens = WhitespaceTokenizer().tokenize(text)
tokens = [token for token in tokens if token not in stop_words]

In [17]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lems = [lemmatizer.lemmatize(token) for token in tokens]
lems[:40]

['chapter',
 'rabbit',
 'holealice',
 'beginning',
 'get',
 'tired',
 'sitting',
 'sister',
 'thebank',
 'nothing',
 'twice',
 'peeped',
 'intothe',
 'book',
 'sister',
 'reading',
 'picture',
 'orconversations',
 'use',
 'book',
 'thought',
 'alice',
 'without',
 'picture',
 'conversation',
 'considering',
 'mind',
 'well',
 'could',
 'thehot',
 'day',
 'made',
 'feel',
 'sleepy',
 'stupid',
 'whether',
 'pleasure',
 'ofmaking',
 'daisy',
 'chain']

**II. Find Top 10 most important (for example, in terms of TF-IDF metric) words from each chapter in the text (not "Alice"). How would you name each chapter according to the identified tokens?**

In [59]:
"""Creating a dict of chapters"""
chapters, c = {}, 1
for i,lem in enumerate(lems):
  if lem == 'chapter' and i != 0:
    chapters[f'Chapter {c}'] = lems[:i-1]
    c+=1
  if c == 12 and lem == 'chapter':
    chapters[f'Chapter {c}'] = lems[i:]

In [130]:
"""Defining function to calculate TF-IDF"""
def TF_IDF(txt):
  TF_IDF, k = {}, 0
  tokens = np.array(txt)
  for j, i in enumerate(np.unique(txt)):
    if i != 'alice':

      _, N = np.unique(txt, return_counts = True)
      TF = N[j]/len(txt)

      for w in range(1,13):
        if w in chapters[f'Chapter {w}']:
          k += 1
      IDF = np.log(12/(k+1))
    TF_IDF[i] = TF*IDF
  sort = sorted(TF_IDF.items(), key=lambda x:x[1], reverse=True)

  return sort[:10]

def show_TFIDF(D):
  for i in range(1,13):
    v1 = TF_IDF(chapters[f'Chapter {i}'])
    print("-"*18, f'\nChapter {i} top:\n')
    for k, v in v1:
      print(k, v, '\n')

In [131]:
show_TFIDF(chapters)

------------------ 
Chapter 1 top:

little 0.036351821418006275 

like 0.02856214539986207 

way 0.02856214539986207 

see 0.02596558672714734 

one 0.023369028054432608 

rabbit 0.023369028054432608 

think 0.023369028054432608 

could 0.018175910709003137 

door 0.018175910709003137 

eat 0.018175910709003137 

------------------ 
Chapter 2 top:

little 0.0388468991629182 

mouse 0.028487726052806674 

said 0.024603036136514857 

like 0.023308139497750915 

way 0.023308139497750915 

one 0.022013242858986974 

oh 0.020718346220223033 

see 0.0194234495814591 

thing 0.0194234495814591 

thought 0.0194234495814591 

------------------ 
Chapter 3 top:

said 0.04747066340520795 

mouse 0.038341689673437186 

little 0.0301256133148435 

one 0.020996639583072744 

like 0.019170844836718593 

thing 0.019170844836718593 

way 0.018257947463541518 

could 0.01734505009036444 

oh 0.01734505009036444 

see 0.016432152717187364 

------------------ 
Chapter 4 top:

said 0.0417631369712269 

li

**Proposed chapter's names:**


1.   Little world
2.   Mouse
3.   Getting to know each other
4.   Rabbit on the way
5.   Time 
6.   Cat
7.   Adventure of the friends Part I
9.   ...Part II
10.  ...Part III
11.  ...Part IV
11.  Adventure of the friends Part V
12.  The battle






**III. Find the Top 10 most used verbs in sentences with Alice. What does Alice do most often?**

In [136]:
sentences = re.split(r"[.?!]", raw_text)
alice = []
for s in sentences:
  if 'Alice' in s:
    alice.append(s)

alice = str(alice).lower() #to lower-case
alice = re.sub("\d+", "", str(alice))

In [156]:
import spacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")
txt = nlp(alice)

verbs = []
for token in txt:
    if token.pos_ == "VERB":
        verbs.append(token.text)
top = Counter(verbs).most_common(10)
for v, n in top:
  print('Word:', v, '\t\t', 'Number of times:', n)

Word: said 		 Number of times: 166
Word: thought 		 Number of times: 53
Word: alice 		 Number of times: 34
Word: went 		 Number of times: 26
Word: ’s 		 Number of times: 26
Word: had 		 Number of times: 25
Word: say 		 Number of times: 21
Word: see 		 Number of times: 21
Word: know 		 Number of times: 21
Word: do 		 Number of times: 20
